In [ ]:
# fine_tune_yolov8_widerface_split.py
import os
import shutil
import random
from pathlib import Path

import cv2
from tqdm import tqdm
from ultralytics import YOLO
from ultralytics.utils import LOGGER

In [ ]:
WIDER_ROOT = Path(r"D:\sangita-mam\assignment-07\WIDER_ROOT")  
YOLO_ROOT = Path(r"D:\sangita-mam\assignment-07\widerface_yolo")
MODEL_WEIGHTS = r"D:\sangita-mam\assignment-07\yolov8n.pt"  

In [ ]:
# Split ratio for train/val
VAL_FRACTION = 0.10  # 10% validation

# Filtering (optional)
SKIP_INVALID = True      # ignore invalid==1 boxes
MIN_BOX_SIZE = 10        # ignore very tiny boxes (<10 px)
ALLOW_EMPTY_LABEL = True # write empty .txt if all boxes filtered (keeps image usable)

# Training config
IMGSZ = 640
EPOCHS = 50
BATCH = 4
DEVICE = 0     # GPU index (e.g., 0) or "cpu"
WORKERS = 0    # Windows/Jupyter-friendly
AMP = True     # mixed precision to save VRAM

In [ ]:
def ensure_dirs():
    for split in ["train", "val"]:
        (YOLO_ROOT / "images" / split).mkdir(parents=True, exist_ok=True)
        (YOLO_ROOT / "labels" / split).mkdir(parents=True, exist_ok=True)

In [ ]:
def read_wider_train_list(wider_root: Path):
    """
    Parse wider_face_train_bbx_gt.txt -> list of relative image paths.
    """
    ann_file = wider_root / "wider_face_split" / "wider_face_train_bbx_gt.txt"
    if not ann_file.exists():
        raise FileNotFoundError(f"Missing annotation file: {ann_file}")

    img_list = []
    with open(ann_file, "r") as f:
        lines = [l.strip() for l in f.readlines()]

    i = 0
    while i < len(lines):
        rel_img_path = lines[i]; i += 1
        n = int(lines[i]); i += 1
        i += n  # skip bbox lines
        img_list.append(rel_img_path)

    return img_list

In [ ]:
def split_list(items, val_fraction=0.1, seed=42):
    random.Random(seed).shuffle(items)
    n_total = len(items)
    n_val = int(round(n_total * val_fraction))
    val_set = set(items[:n_val])
    train_list = [x for x in items if x not in val_set]
    val_list = [x for x in items if x in val_set]
    return train_list, val_list

def copy_image(src: Path, dst: Path):
    dst.parent.mkdir(parents=True, exist_ok=True)
    if not dst.exists():
        shutil.copy2(src, dst)

In [ ]:
def convert_and_write_labels_for_split(wider_root: Path, rel_list, split_name: str):
    """
    Stream through the annotation file again; convert only images in rel_list for this split.
    Writes images to YOLO_ROOT/images/<split_name> and labels to YOLO_ROOT/labels/<split_name>.
    """
    img_src_root = wider_root / "WIDER_train" / "images"
    ann_file = wider_root / "wider_face_split" / "wider_face_train_bbx_gt.txt"

    if not ann_file.exists():
        raise FileNotFoundError(f"Missing annotation file: {ann_file}")
    rel_set = set(rel_list)

    with open(ann_file, "r") as f:
        lines = [l.strip() for l in f.readlines()]

    i = 0
    pbar = tqdm(total=len(rel_list), desc=f"Converting {split_name}")
    while i < len(lines):
        rel_img = lines[i]; i += 1
        n = int(lines[i]); i += 1

        # If this image isn't in the split, skip its bbox lines
        if rel_img not in rel_set:
            i += n
            continue

        # Paths
        src_img = img_src_root / rel_img
        dst_img = YOLO_ROOT / "images" / split_name / Path(rel_img).name
        dst_lbl = YOLO_ROOT / "labels" / split_name / (Path(rel_img).stem + ".txt")

        # Copy image
        copy_image(src_img, dst_img)

        # Read image to get size
        img = cv2.imread(str(dst_img))
        if img is None:
            # consume bbox lines and skip
            i += n
            pbar.update(1)
            continue
        h, w = img.shape[:2]

        yolo_lines = []
        # Each bbox line: x y w h blur expression illumination invalid occlusion pose
        for _ in range(n):
            parts = lines[i].split()
            i += 1
            if len(parts) < 10:
                continue

            x, y, bw, bh = map(float, parts[:4])
            blur, expr, illum, invalid, occ, pose = map(int, parts[4:10])

            if SKIP_INVALID and invalid == 1:
                continue
            if bw <= 0 or bh <= 0:
                continue
            if MIN_BOX_SIZE and (bw < MIN_BOX_SIZE or bh < MIN_BOX_SIZE):
                continue

            # Convert to YOLO normalized (cx, cy, w, h)
            cx = (x + bw / 2.0) / w
            cy = (y + bh / 2.0) / h
            nw = bw / w
            nh = bh / h

            # Clip to [0,1]
            cx = min(max(cx, 0.0), 1.0)
            cy = min(max(cy, 0.0), 1.0)
            nw = min(max(nw, 0.0), 1.0)
            nh = min(max(nh, 0.0), 1.0)

            yolo_lines.append(f"0 {cx:.6f} {cy:.6f} {nw:.6f} {nh:.6f}")

        # Write label file (possibly empty)
        dst_lbl.parent.mkdir(parents=True, exist_ok=True)
        if yolo_lines or ALLOW_EMPTY_LABEL:
            with open(dst_lbl, "w") as lf:
                lf.write("\n".join(yolo_lines))
        else:
            # Remove the image if we keep no label (optional)
            try:
                os.remove(dst_img)
            except Exception:
                pass

        pbar.update(1)

    pbar.close()

In [ ]:
def write_dataset_yaml():
    yaml_text = f"""# WIDER FACE (converted to YOLO format)
path: {YOLO_ROOT.as_posix()}
train: images/train
val: images/val

names:
  0: face
"""
    yaml_path = YOLO_ROOT / "widerface.yaml"
    with open(yaml_path, "w") as f:
        f.write(yaml_text)
    return yaml_path

In [ ]:
# ----- robust scalar helper for epoch callback -----
def _safe_scalar(x):
    """Return a Python float from tensor/dict/list scalars; else None."""
    try:
        import torch
        if isinstance(x, torch.Tensor):
            return x.detach().float().mean().item()
        if isinstance(x, (list, tuple)):
            vals = [_safe_scalar(v) for v in x]
            vals = [v for v in vals if v is not None]
            return sum(vals) / max(1, len(vals)) if vals else None
        if isinstance(x, dict):
            vals = [_safe_scalar(v) for v in x.values()]
            vals = [v for v in vals if v is not None]
            return sum(vals) / max(1, len(vals)) if vals else None
        return float(x)
    except Exception:
        return None

In [ ]:
# ----- robust scalar helper for epoch callback -----
def _safe_scalar(x):
    """Return a Python float from tensor/dict/list scalars; else None."""
    try:
        import torch
        if isinstance(x, torch.Tensor):
            return x.detach().float().mean().item()
        if isinstance(x, (list, tuple)):
            vals = [_safe_scalar(v) for v in x]
            vals = [v for v in vals if v is not None]
            return sum(vals) / max(1, len(vals)) if vals else None
        if isinstance(x, dict):
            vals = [_safe_scalar(v) for v in x.values()]
            vals = [v for v in vals if v is not None]
            return sum(vals) / max(1, len(vals)) if vals else None
        return float(x)
    except Exception:
        return None

In [ ]:
def train_yolov8(yaml_path):
    print("\n[Training] Starting YOLOv8 fine-tuning...")
    LOGGER.setLevel("INFO")
    model = YOLO(MODEL_WEIGHTS)

    # Robust epoch-end print (works across Ultralytics versions)
    def on_epoch_end(trainer):
        epoch = trainer.epoch + 1
        epochs = trainer.epochs
        tloss = _safe_scalar(getattr(trainer, "tloss", None))
        if tloss is None:
            print(f"[Epoch {epoch}/{epochs}] (loss unavailable)")
        else:
            print(f"[Epoch {epoch}/{epochs}] total_loss={tloss:.4f}")

    model.add_callback("on_train_epoch_end", on_epoch_end)

    results = model.train(
        data=str(yaml_path),
        imgsz=IMGSZ,
        epochs=EPOCHS,
        batch=BATCH,
        device=DEVICE,
        workers=WORKERS,   # Windows/Jupyter-friendly
        project="runs_widerface_split",
        name=Path(MODEL_WEIGHTS).stem + f"_faces_{IMGSZ}",
        verbose=True,
        plots=True,
        amp=AMP
    )
    print("[Training] Done.")
    return results

In [ ]:
def validate_and_sample(best_weights_path, yaml_path, n_samples=20):
    print("\n[Validate] Running validation on val split...")
    model = YOLO(best_weights_path)
    model.val(data=str(yaml_path), imgsz=IMGSZ, device=DEVICE, workers=WORKERS)

    # Save a handful of annotated validation images for visual check
    val_dir = YOLO_ROOT / "images" / "val"
    imgs = []
    for root, _, files in os.walk(val_dir):
        for f in files:
            if f.lower().endswith((".jpg", ".jpeg", ".png")):
                imgs.append(os.path.join(root, f))
    random.shuffle(imgs)
    imgs = imgs[:n_samples] if len(imgs) > n_samples else imgs

    if imgs:
        print(f"[Predict] Saving {len(imgs)} annotated validation images...")
        model.predict(
            source=imgs,
            conf=0.25,
            imgsz=IMGSZ,
            device=DEVICE,
            save=True,        # save annotated images
            save_txt=False,
            save_conf=True,
            project="runs_widerface_split",
            name="val_samples",   # runs_widerface_split/val_samples/
            exist_ok=True,
            workers=WORKERS
        )
        print("Annotated samples saved to: runs_widerface_split/val_samples/")
    else:
        print("No images found in val set to sample.")

In [ ]:

if __name__ == "__main__":
    # 0) Quick sanity checks
    assert (WIDER_ROOT / "WIDER_train" / "images").exists(), \
        f"Missing: {WIDER_ROOT}/WIDER_train/images"
    assert (WIDER_ROOT / "wider_face_split" / "wider_face_train_bbx_gt.txt").exists(), \
        f"Missing: {WIDER_ROOT}/wider_face_split/wider_face_train_bbx_gt.txt"

    # 1) Make output dirs
    ensure_dirs()

    # 2) Read list of WIDER_train images from the annotation file
    all_train_images = read_wider_train_list(WIDER_ROOT)
    print(f"Found {len(all_train_images)} training images listed in WIDER FACE.")

    # 3) Split into train/val
    train_list, val_list = split_list(all_train_images, VAL_FRACTION, seed=123)
    print(f"Split -> train: {len(train_list)}, val: {len(val_list)}")

    # 4) Convert and copy for train and val
    convert_and_write_labels_for_split(WIDER_ROOT, train_list, "train")
    convert_and_write_labels_for_split(WIDER_ROOT, val_list, "val")

    # 5) Write dataset YAML
    yaml_path = write_dataset_yaml()
    print(f"Dataset YAML written to: {yaml_path}")

    # 6) Train
    results = train_yolov8(yaml_path)
    # Locate best weights across Ultralytics versions
    try:
        best_weights = results.best  # Ultralytics >=8.1 returns path in .best
    except Exception:
        exp_dir = Path(getattr(results, "save_dir", "runs_widerface_split"))
        cand = list((exp_dir / "weights").glob("best*.pt"))
        if not cand:
            # try searching all subdirs of project
            cand = list(Path("runs_widerface_split").rglob("best*.pt"))
        best_weights = cand[0].as_posix() if cand else None
    print(f"Best weights: {best_weights}")

    # 7) Validate & save example predictions on val images
    if best_weights is not None:
        validate_and_sample(best_weights, yaml_path, n_samples=20)
    else:
        print("WARNING: best weights not found; skipping prediction.")

    print("\nAll done. YOLO dataset root:", YOLO_ROOT.resolve())


Found 12880 training images listed in WIDER FACE.
Split -> train: 11592, val: 1288


Converting val: 100%|██████████| 1288/1288 [00:14<00:00, 86.64it/s]


Dataset YAML written to: D:\sangita-mam\assignment-07\widerface_yolo\widerface.yaml

[Training] Starting YOLOv8 fine-tuning...
New https://pypi.org/project/ultralytics/8.3.177 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.175  Python-3.10.11 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:\sangita-mam\assignment-07\widerface_yolo\widerface.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mas

train: Scanning D:\sangita-mam\assignment-07\widerface_yolo\labels\train.cache... 11592 images, 19 backgrounds, 0 corrupt: 100%|██████████| 11592/11592 [00:00<?, ?it/s]

val: Fast image access  (ping: 0.00.0 ms, read: 856.8474.6 MB/s, size: 103.8 KB)


...

                   all       1288      12515      0.876      0.699      0.799      0.455
Speed: 0.3ms preprocess, 6.5ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs_widerface_split\yolov8n_faces_640
[Training] Done.
Best weights: runs_widerface_split/yolov8n_faces_640/weights/best.pt

[Validate] Running validation on val split...
Ultralytics 8.3.175  Python-3.10.11 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4096MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 968.5286.3 MB/s, size: 116.2 KB)


val: Scanning D:\sangita-mam\assignment-07\widerface_yolo\labels\val.cache... 1288 images, 2 backgrounds, 0 corrupt: 100%|██████████| 1288/1288 [00:00<?, ?it/s]

val: D:\sangita-mam\assignment-07\widerface_yolo\images\val\2_Demonstration_Protesters_2_231.jpg: 1 duplicate labels removed



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 81/81 [00:21<00:00,  3.80it/s]


                   all       1288      12515      0.876      0.699        0.8      0.457
Speed: 0.3ms preprocess, 6.4ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs\detect\val
[Predict] Saving 20 annotated validation images...

0: 640x640 1 face, 7.9ms
1: 640x640 1 face, 7.9ms
2: 640x640 1 face, 7.9ms
3: 640x640 5 faces, 7.9ms
4: 640x640 3 faces, 7.9ms
5: 640x640 6 faces, 7.9ms
6: 640x640 135 faces, 7.9ms
7: 640x640 5 faces, 7.9ms
8: 640x640 1 face, 7.9ms
9: 640x640 2 faces, 7.9ms
10: 640x640 4 faces, 7.9ms
11: 640x640 136 faces, 7.9ms
12: 640x640 1 face, 7.9ms
13: 640x640 2 faces, 7.9ms
14: 640x640 2 faces, 7.9ms
15: 640x640 4 faces, 7.9ms
16: 640x640 1 face, 7.9ms
17: 640x640 6 faces, 7.9ms
18: 640x640 4 faces, 7.9ms
19: 640x640 2 faces, 7.9ms
Speed: 3.2ms preprocess, 7.9ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs_widerface_split\val_samples
Annotated samples saved to: runs_widerface_split/val_samples/

All don